In [1]:
! pip install transformers

     |████████████████████████████████| 1.8MB 8.5MB/s 
     |████████████████████████████████| 2.9MB 17.7MB/s 
     |████████████████████████████████| 890kB 59.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=abb091d2d3e4d58944f14a5127ab2c0ac0090ced6ea78c11783535a8fef4b1f3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_table('../content/news_train.txt',header=None)

In [5]:
df.columns = ['sentence','label']

In [6]:
df.head()

,sentence,label
0,中华女子学院：本科层次仅1专业招男生,3
1,两天价网站背后重重迷雾：做个网站究竟要多少钱,4
2,东5环海棠公社230-290平2居准现房98折优惠,1
3,卡佩罗：告诉你德国脚生猛的原因 不希望英德战踢点球,7
4,82岁老太为学生做饭扫地44年获授港大荣誉院士,5


In [7]:
# finance
# realty
# stocks
# education
# science
# society
# politics
# sports
# game
# entertainment
df.label.value_counts()

9    18000
8    18000
7    18000
6    18000
5    18000
4    18000
3    18000
2    18000
1    18000
0    18000
Name: label, dtype: int64

In [38]:
# 得到資料集中最大的句子長度
max_len = max([len(s) for s in df.sentence])

In [18]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer

from torch.utils.data import TensorDataset

import transformers
from transformers import BertForSequenceClassification

# pytorch須將資料放在DataLoader
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# 與optimizer有關
from transformers import AdamW, get_linear_schedule_with_warmup
# 評估模型
from sklearn.metrics import f1_score
# 輸出畫面優化
from tqdm.notebook import tqdm

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df['sentence'], df['label'], test_size = 0.2)

In [39]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-chinese')

In [40]:
encoded_data_train = tokenizer.batch_encode_plus(
    x_train, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=max_len, # 設定為資料集的最大句子長度或直接給固定的值(可能之後用在新資料比較方便)
    truncation=True, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    x_val, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=max_len,
    truncation=True, 
    return_tensors='pt'
)

In [41]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val.values)


# Pytorch TensorDataset Instance
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [42]:
# initializing the model

model = transformers.BertForSequenceClassification.from_pretrained("bert-base-chinese",
                                   num_labels=10,
                                   output_attentions=False,
                                   output_hidden_states=False)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [43]:
# 設定batch_size
batch_size = 32

# RandomSampler隨機抽出的資料
dataloader_train = DataLoader(dataset_train, 
                sampler=RandomSampler(dataset_train), 
                batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                  sampler=SequentialSampler(dataset_val), 
                  batch_size=batch_size)

In [44]:
# Adam algorithm with weight decay fix 
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 3
# 設定learning rate變化的時程
scheduler = get_linear_schedule_with_warmup(optimizer, 
                       num_warmup_steps=0,
                       num_training_steps=len(dataloader_train)*epochs)

In [45]:
# 建立評估模型的function
class_dict ={
        0:"finance",
        1:"realty",
        2:"stocks",
        3:"education",
        4:"science",
        5:"society",
        6:"politics",
        7:"sports",
        8:"game",
        9:"entertainment"
        }


def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {class_dict[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [46]:
# 將模型放到gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [47]:
# 使用驗證資料集評估模型
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':   batch[0],
              'attention_mask': batch[1],
              'labels':     batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [48]:
# 正式訓練模型
# tqdm優化輸出結果
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':   batch[0],
              'attention_mask': batch[1],
                  'labels': batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.3112394448655347
Validation loss: 0.2115081021843685
F1 Score (Weighted): 0.9330555002245406



Epoch 2
Training loss: 0.1663974166414183
Validation loss: 0.1990831138384011
F1 Score (Weighted): 0.9384134878964226



Epoch 3
Training loss: 0.11794366048224685
Validation loss: 0.20796099427352763
F1 Score (Weighted): 0.9407730902869337



In [25]:
torch.save(model.state_dict(), '../content/news_ch_BERT_epoch_3.model')

In [26]:
df_test = pd.read_table('../content/news_test.txt',header=None)

In [27]:
df_test.columns = ['sentence','label']

In [49]:
encoded_data_test = tokenizer.batch_encode_plus(
    df_test.sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=max_len,
    truncation=True, 
    return_tensors='pt'
)

# 將資料合併成dataset，方便之後匯入至模型
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df_test.label.values)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [50]:
dataloader_test = DataLoader(dataset_test, 
                sampler=SequentialSampler(dataset_test), 
                batch_size=batch_size)

In [51]:
_, predictions, true_vals = evaluate(dataloader_test)

In [52]:
f1_score_func(predictions, true_vals)

0.9445903371175914

In [53]:
accuracy_per_class(predictions, true_vals)

Class: finance
Accuracy: 941/1000

Class: realty
Accuracy: 961/1000

Class: stocks
Accuracy: 898/1000

Class: education
Accuracy: 967/1000

Class: science
Accuracy: 910/1000

Class: society
Accuracy: 953/1000

Class: politics
Accuracy: 933/1000

Class: sports
Accuracy: 978/1000

Class: game
Accuracy: 936/1000

Class: entertainment
Accuracy: 969/1000



In [63]:
from sklearn.metrics import confusion_matrix
predicted_category = [class_dict[np.argmax(x)] for x in predictions]
true_category = [class_dict[x] for x in true_vals]

confusion_mat = confusion_matrix(y_true = true_category, y_pred = predicted_category, labels=list(class_dict.values()))

In [64]:
df = pd.DataFrame(confusion_mat, index = class_dict.values(),columns = class_dict.values())
df

,finance,realty,stocks,education,science,society,politics,sports,game,entertainment
finance,941,11,28,1,6,5,6,1,1,0
realty,10,961,6,2,2,6,6,1,1,5
stocks,29,17,898,0,25,1,24,1,3,2
education,2,0,0,967,1,13,10,1,0,6
science,7,2,14,3,910,14,14,1,25,10
society,1,6,1,11,3,953,13,0,3,9
politics,5,4,18,7,15,14,933,1,0,3
sports,2,2,2,0,1,3,3,978,0,9
game,0,2,2,2,42,5,1,2,936,8
entertainment,0,0,1,3,8,5,2,8,4,969


In [37]:
# load model
model = BertForSequenceClassification.from_pretrained("bert-base-chinese",
                            num_labels=10,
                            output_attentions=False,
                            output_hidden_states=False)

model.to(device)
model.load_state_dict(torch.load('../content/news_ch_BERT_epoch_3.model', map_location=torch.device('cuda')))

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>